#### Import the local navigation file 

In [ ]:
from local_navigation import *

#### Connect the Thymio

In [ ]:
try:
    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()
    print("Thymio connected")
except:
    print("Thymio not connected")

#### Local parameters

In [ ]:
motor_speed = 100   # speed of the robot 
LTobst = 5      # low obstacle threshold to switch state 1->0
HTobst = 13    # high obstacle threshold to switch state 0->1
obst_gain = 15  # /100 (actual gain: 15/100=0.15)

state = 0      # Actual state of the robot: 0->global navigation, 1->obstacle avoidance
case = 0       # Actual case of obstacle avoidance: 0-> left obstacle, 1-> right obstacle, 2-> obstacle in front  
side = bool(random.randint(0,1)) # we randomly generate the first bypass choice of the robot when he encounters an obstacle in front of him  

rotation_time = 1  # 1ms time Thymio takes to rotate 
step_back_time = 1 # 1ms time Thymio takes to move backwards 

prox_horizontal = [0, 0, 0, 0, 0] #initialize the value of the proximal sensors to 0

#### Local main 

In [ ]:
async def prog():
    while True:
        await node.wait_for_variables({"prox.horizontal"})
        for i in range(0, 5):
            prox_horizontal[i]=list(node.v.prox.horizontal)[i]  
        await client.sleep(0.01)
        obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]

        await avoid_obstacles(node,client,state,side,obst,HTobst, LTobst, motor_speed, step_back_time,obst_gain, rotation_time)
        
await prog()